In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# ResNet
잔차 네트워크를 사용<br>
처음 구상된 ResNet은 152개의 층으로 구성된 깊은 CNN인데 이런 깊은 네트워크를 훈련시킬 수 있는 핵심 요소는 스킵 연결(skip connection, shortcut connection)이다. 즉 어떤 층에 주입되는 신호가 상위 층의 출력에 더해지는 형태이다.

신경망을 훈련시킬 때는 목적 함수 $h(x)$를 훈련시키는 것이 목표이다. 만일 입력 x를 네트워크의 출력에 더한다면 네트워크는 $h(x)$ 대신 $f(x) = h(x) - x$를 학습하게 된다. 이를 잔차 학습이라한다.

일반적인 신경망을 초기화할 때는 가중치가 0에 가깝기 때문에 네트워크도 0에 가까운 값을 출력한다. 스킵 연결을 추가하면 네트워크는 입력값과 같은 값을 출력하게 된다. 즉, 초기 단계에서는 항등함수에 가깝게 모델링된다. 목적 함수가 항등 함수에 가깝다면 훈련 속도가 매우 빨라지게 된다.<br>
또 스킵 연결을 많이 추가하면 일부 층이 아직 학습되지 않았더라도 네트워크는 훈련을 시작할 수 있다. 스킵 연결 덕분에 입력 신호가 전체 네트워크에 손쉽게 영향을 미치게 된다. <br>심층 잔차 네트워크는 스킵 연결을 가진 작은 신경망인 잔차 유닛(RU,residual unit)을 쌓은 것으로 볼 수 있다.

## 구현
ResNet - 34 모델

In [ ]:
# ResidualUnit 층 구성(p571 그림 14-18 구현, main_layers가 오른쪽 모듈, skip_layers가 왼쪽 모듈)
class ResidualUnit(keras.layers.Layer):
  def __init__(self,filters,strides=1,activation='relu',**kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
                        keras.layers.Conv2D(filters,3,strides=strides,
                                            padding='same',use_bias=False),
                        keras.layers.BatchNormalization(),
                        self.activation,
                        keras.layers.Conv2D(filters,3,strides=1,
                                            padding='same',use_bias=False),
                        keras.layers.BatchNormalization()]
    self.skip_layers = []
    if strides > 1:
      self.skip_layers =[
                         keras.layers.Conv2D(filters,1,strides=strides,
                                             padding='same',use_bias=False),
                         keras.layers.BatchNormalization()]
  def call(self,inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)

    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(Z+skip_Z)

In [ ]:
# ResNet-34 모델
model = keras.Sequential()
model.add(keras.layers.Conv2D(64,7,strides=2,input_shape=[224,224,3],padding='same',use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))
# 처음 3개 RU는 64개의 필터, 그다음 4개는 128개를 가지는 식
prev_filters = 64
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
  strides = 1 if filters == prev_filters else 2
  model.add(ResidualUnit(filters.strides=strides))
  prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation='softmax'))

# 케라스에서 제공하는 사전훈련된 모델 사용하기

In [ ]:
# keras.applications 패키지에 사전훈련된 모델들이 내장되어있다.
# ResNet - 50 모델을 만들고 이미지넷 데이터셋에서 사전훈련된 가중치를 다운로드한다.
model = keras.applications.resnet50.ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
# model.summary()로 확인해보면 224x224 픽셀 크기의 이미지를 기대하는데 다른 크기라면 tf.image.resize() 함수로 이미지의 크기를 바꿔야한다.
images_resized = tf.image.resize(images,[224,224])

In [ ]:
# 사전 훈련된 모델은 이미지가 0~1 혹은 -1~1 사이의 입력을 기대한다. 이를 위해 모델마다 이미지를 전처리해주는 preprocess_input()함수가 제공된다.
# 픽셀 값이 0~255 사이로 가정하기 때문에 0~1 사이로 바뀐 이미지는 255를 곱해주어야한다.
inputs = keras.applications.resnet50.preprocess_input(images_resized*255)